In [ ]:
%load_ext tensorboard

In [1]:
import evaluate
import numpy as np
import torch
from datasets import Dataset, load_dataset
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (
    AutoModelForSequenceClassification,
    CanineModel,
    CanineTokenizer,
    Trainer,
    TrainingArguments,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2023-03-21 10:37:30.900391: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-21 10:37:32.372042: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/matias/RECVIS:
2023-03-21 10:37:32.372162: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/matias/RECVIS:
2023-03-21 10:37:32.372171: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-T

In [38]:
# train_dataset = load_dataset("amazon_reviews_multi", split="train").with_format(
#     "torch", device=device
# )
# valid_dataset = load_dataset("amazon_reviews_multi", split="validation").with_format(
#     "torch", device=device
# )
# test_dataset = load_dataset("amazon_reviews_multi", split="test").with_format(
#     "torch", device=device
# )

train_dataset_tokenized = (
    load_dataset(
        "json", data_files={"train": "ARC/train_dataset_tokenized.jsonl"}, split="train"
    )
    .with_format(
        "torch",  # device=device
    )
    .rename_column("stars", "label")
)
valid_dataset_tokenized = (
    load_dataset(
        "json", data_files={"train": "ARC/valid_dataset_tokenized.jsonl"}, split="train"
    )
    .with_format(
        "torch",  # device=device
    )
    .rename_column("stars", "label")
)
test_dataset_tokenized = (
    load_dataset(
        "json", data_files={"train": "ARC/test_dataset_tokenized.jsonl"}, split="train"
    )
    .with_format(
        "torch",  # device=device
    )
    .rename_column("stars", "label")
)


Using custom data configuration default-0e4484f09108c51b
Found cached dataset json (/home/matias/.cache/huggingface/datasets/json/default-0e4484f09108c51b/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)
Using custom data configuration default-aa8492bc82944451
Found cached dataset json (/home/matias/.cache/huggingface/datasets/json/default-aa8492bc82944451/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)
Using custom data configuration default-f2fdf9e761c24ab9
Found cached dataset json (/home/matias/.cache/huggingface/datasets/json/default-f2fdf9e761c24ab9/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


In [8]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5
batches_per_epoch = len(train_dataset_tokenized) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

2023-03-20 11:29:36.476764: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/matias/RECVIS:
2023-03-20 11:29:36.480293: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)


In [29]:
def tokenization(sample):
    inputs = tokenizer(
        sample["review_body"],
        padding="max_length",
        truncation=True,
        return_tensors="pt",
        max_length=2048,
    )
    inputs["stars"] = sample["stars"]
    return inputs


train_dataset_tokenized = train_dataset.map(
    tokenization, batched=True, desc="Tokenizing", num_proc=8
)
valid_dataset_tokenized = valid_dataset.map(
    tokenization, batched=True, desc="Tokenizing", num_proc=8
)
test_dataset_tokenized = test_dataset.map(
    tokenization, batched=True, desc="Tokenizing", num_proc=8
)


Tokenizing #0:   0%|          | 0/150 [00:00<?, ?ba/s]

Tokenizing #1:   0%|          | 0/150 [00:00<?, ?ba/s]

Tokenizing #2:   0%|          | 0/150 [00:00<?, ?ba/s]

Tokenizing #3:   0%|          | 0/150 [00:00<?, ?ba/s]

Tokenizing #4:   0%|          | 0/150 [00:00<?, ?ba/s]

Tokenizing #5:   0%|          | 0/150 [00:00<?, ?ba/s]

Tokenizing #6:   0%|          | 0/150 [00:00<?, ?ba/s]

Tokenizing #7:   0%|          | 0/150 [00:00<?, ?ba/s]

{'review_id': ['de_0394461', 'zh_0794564', 'zh_0614410', 'de_0555964'],
 'product_id': ['product_de_0820593',
  'product_zh_0496479',
  'product_zh_0093196',
  'product_de_0163377'],
 'reviewer_id': ['reviewer_de_0452612',
  'reviewer_zh_0393922',
  'reviewer_zh_0277078',
  'reviewer_de_0391880'],
 'stars': tensor([4, 4, 4, 2]),
 'review_body': ['Gute Material und zum trainieren, warm machen sehr schön',
  '手机很好看，功能挺多，就是说明书不够详细，镜头像素比较低，孩子的玩具。',
  '不知道真假，虽然还是不放心，但是还是相信亚马逊吧',
  'Sturmpfeil ist zum einen kleiner als Ohnezahn, was in der Serie anders ist, und meine Kinder beim Spielen stört. Zum anderen, und das nervt richtig, verliert der Drache ständig seine Flügel! Man kann sie kaum bewegen, manchmal reicht schon ein leichter Stoß. Wir haben noch andere playmobil Drachen und da war es genau das gleiche. Der Mechanismus scheint nicht sehr ausgereift zu sein. Wir haben hier jedenfalls viel Frust deswegen und die Drachen werden kaum bespielt.'],
 'review_title': ['Preis, Qualität top',
  '

In [33]:
train_dataloader = DataLoader(
    train_dataset_tokenized, batch_size=4, num_workers=4, shuffle=True
)
next(iter(train_dataloader))


{'review_id': ['fr_0308102', 'de_0369534', 'ja_0212637', 'fr_0961610'],
 'product_id': ['product_fr_0552827',
  'product_de_0469030',
  'product_ja_0110104',
  'product_fr_0210630'],
 'reviewer_id': ['reviewer_fr_0477995',
  'reviewer_de_0099631',
  'reviewer_ja_0773866',
  'reviewer_fr_0922413'],
 'stars': tensor([5, 1, 2, 1]),
 'review_body': ['Conforme à mes attentes. Reçu dans les temps.',
  'Ist Billig und keine klare Sicht nicht gud',
  'ムラもあるし、そもそも安定した吹き出しをしないため、塗装には向きません。 美容用として化粧水など、粘度が極低いものには良いです。',
  "c'est un produit de très mauvaise qualité et le descriptif ne correspond nullement au produit présenté, évitez d'en acheter car photos trompeuses"],
 'review_title': ['Produit conforme',
  'Digitale Unisex-Led Armbanduhr',
  '塗装には向きません',
  'arnaque'],
 'language': ['fr', 'de', 'ja', 'fr'],
 'product_category': ['wireless', 'watch', 'toy', 'personal_care_appliances'],
 'input_ids': tensor([[57344,    67,   111,  ...,     0,     0,     0],
         [57344,    73,   115,  ...,    

In [41]:
model


CanineModel(
  (char_embeddings): CanineEmbeddings(
    (HashBucketCodepointEmbedder_0): Embedding(16384, 96)
    (HashBucketCodepointEmbedder_1): Embedding(16384, 96)
    (HashBucketCodepointEmbedder_2): Embedding(16384, 96)
    (HashBucketCodepointEmbedder_3): Embedding(16384, 96)
    (HashBucketCodepointEmbedder_4): Embedding(16384, 96)
    (HashBucketCodepointEmbedder_5): Embedding(16384, 96)
    (HashBucketCodepointEmbedder_6): Embedding(16384, 96)
    (HashBucketCodepointEmbedder_7): Embedding(16384, 96)
    (char_position_embeddings): Embedding(16384, 768)
    (token_type_embeddings): Embedding(16, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (initial_char_encoder): CanineEncoder(
    (layer): ModuleList(
      (0): CanineLayer(
        (attention): CanineAttention(
          (self): CanineSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Li

In [48]:
class ClassificationHead(nn.Module):
    def __init__(self, dense_layers_sizes=[32, 5], canine_model=model):
        super(ClassificationHead, self).__init__()
        self.dense_layers_sizes = dense_layers_sizes
        self.dense_layers = [nn.LazyLinear(size) for size in self.dense_layers_sizes]
        self.canine_model = canine_model

    def forward(self, x):
        out = self.canine_model(
            input_ids=x["input_ids"],
            token_type_ids=x["token_type_ids"],
            attention_mask=x["attention_mask"],
        )  # forward pass
        out = out.pooler_output
        # sequence_output = out.last_hidden_state

        for linear in self.dense_layers:
            out = linear(out)
            out = nn.ReLU()(out)
        return out


clf = ClassificationHead()
clf.canine_model


/home/matias/.pyenv/versions/3.9.8/envs/MVA/lib/python3.9/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


CanineModel(
  (char_embeddings): CanineEmbeddings(
    (HashBucketCodepointEmbedder_0): Embedding(16384, 96)
    (HashBucketCodepointEmbedder_1): Embedding(16384, 96)
    (HashBucketCodepointEmbedder_2): Embedding(16384, 96)
    (HashBucketCodepointEmbedder_3): Embedding(16384, 96)
    (HashBucketCodepointEmbedder_4): Embedding(16384, 96)
    (HashBucketCodepointEmbedder_5): Embedding(16384, 96)
    (HashBucketCodepointEmbedder_6): Embedding(16384, 96)
    (HashBucketCodepointEmbedder_7): Embedding(16384, 96)
    (char_position_embeddings): Embedding(16384, 768)
    (token_type_embeddings): Embedding(16, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (initial_char_encoder): CanineEncoder(
    (layer): ModuleList(
      (0): CanineLayer(
        (attention): CanineAttention(
          (self): CanineSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Li

In [49]:
clf(next(iter(train_dataloader)))


: 

: 

In [37]:
model = CanineModel.from_pretrained("google/canine-c")
tokenizer = CanineTokenizer.from_pretrained("google/canine-c")
inputs = ["Life is like a box of chocolates.", "You never know what you gonna get."]
encoding = tokenizer(
    inputs, padding="max_length", truncation=True, return_tensors="pt", max_length=2048
)
outputs = model(**encoding)  # forward pass
pooled_output = outputs.pooler_output
sequence_output = outputs.last_hidden_state


Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


In [38]:
encoding.keys()
input_ids token_type_ids attention_mask

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [39]:
encoding.attention_mask.float().sum(1)


tensor([35., 36.])

In [3]:
label2id = {"0": 0, "1": 1, "2": 2, "3": 3, "4": 4}

#id2label = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5}
#label2id = {1: 0, 2: 1, 3: 2, 4: 3, 5: 4}

id2label = {}
for label, _id in label2id.items():
    id2label[_id] = label
id2label

{0: '0', 1: '1', 2: '2', 3: '3', 4: '4'}

In [4]:
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased", num_labels=5, id2label=id2label, label2id=label2id
)


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [5]:
sum(p.numel() for p in model.parameters())

177857285

In [6]:
canine = AutoModelForSequenceClassification.from_pretrained(
    "google/canine-c", num_labels=5, id2label=id2label, label2id=label2id
)


Some weights of CanineForSequenceClassification were not initialized from the model checkpoint at google/canine-c and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
sum(p.numel() for p in canine.parameters())

132086789

In [8]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [9]:
canine

CanineForSequenceClassification(
  (canine): CanineModel(
    (char_embeddings): CanineEmbeddings(
      (HashBucketCodepointEmbedder_0): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_1): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_2): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_3): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_4): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_5): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_6): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_7): Embedding(16384, 96)
      (char_position_embeddings): Embedding(16384, 768)
      (token_type_embeddings): Embedding(16, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (initial_char_encoder): CanineEncoder(
      (layer): ModuleList(
        (0): CanineLayer(
          (attention): CanineAttention(
            (self): CanineSelfAttention(
            